## Sentimentanalyse von englischen [Kleidungsbewertungen](https://www.kaggle.com/timoboz/womens-ecommerce-clothing-reviews)
[Tutorial Hilfe](https://www.dataquest.io/blog/tutorial-text-classification-in-python-using-spacy/)

In [27]:
import spacy
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns

In [46]:
data = pd.read_csv('kleidung_reviews.csv')

In [47]:
data.head()

Unnamed: 0  Unnamed: 0.1  Clothing ID  Age                    Title  \
0           0             0          767   33                      NaN   
1           1             1         1080   34                      NaN   
2           2             2         1077   60  Some major design flaws   
3           3             3         1049   50         My favorite buy!   
4           4             4          847   47         Flattering shirt   

                                         Review Text  Rating  Recommended IND  \
0  Absolutely wonderful - silky and sexy and comf...       4                1   
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
3  I love, love, love this jumpsuit. it's fun, fl...       5                1   
4  This shirt is very flattering to all due to th...       5                1   

   Positive Feedback Count   Division Name Department Name Class Name  \
0                        0       Initmates        Intimate  Intimates   
1                        4         General         Dresses    Dresses   
2                        0         General         Dresses    Dresses   
3                        0  General Petite         Bottoms      Pants   
4                        6         General            Tops    Blouses   

                                      Review Text DE  
0    Absolut wundervoll - seidig und sexy und bequem  
1  Liebe dieses Kleid! es ist sooo hübsch. Ich fa...  
2  Ich hatte so große Hoffnungen auf dieses Kleid...  
3  Ich liebe, liebe, liebe diesen Overall. Es mac...  
4  Dieses Shirt ist aufgrund der verstellbaren Fr...

In [48]:
data.dtypes

Unnamed: 0                  int64
Unnamed: 0.1                int64
Clothing ID                 int64
Age                         int64
Title                      object
Review Text                object
Rating                      int64
Recommended IND             int64
Positive Feedback Count     int64
Division Name              object
Department Name            object
Class Name                 object
Review Text DE             object
dtype: object

In [50]:
data['Review Text'] = data['Review Text'].astype(str)
data.head()

Unnamed: 0  Unnamed: 0.1  Clothing ID  Age                    Title  \
0           0             0          767   33                      NaN   
1           1             1         1080   34                      NaN   
2           2             2         1077   60  Some major design flaws   
3           3             3         1049   50         My favorite buy!   
4           4             4          847   47         Flattering shirt   

                                         Review Text  Rating  Recommended IND  \
0  Absolutely wonderful - silky and sexy and comf...       4                1   
1  Love this dress!  it's sooo pretty.  i happene...       5                1   
2  I had such high hopes for this dress and reall...       3                0   
3  I love, love, love this jumpsuit. it's fun, fl...       5                1   
4  This shirt is very flattering to all due to th...       5                1   

   Positive Feedback Count   Division Name Department Name Class Name  \
0                        0       Initmates        Intimate  Intimates   
1                        4         General         Dresses    Dresses   
2                        0         General         Dresses    Dresses   
3                        0  General Petite         Bottoms      Pants   
4                        6         General            Tops    Blouses   

                                      Review Text DE  
0    Absolut wundervoll - seidig und sexy und bequem  
1  Liebe dieses Kleid! es ist sooo hübsch. Ich fa...  
2  Ich hatte so große Hoffnungen auf dieses Kleid...  
3  Ich liebe, liebe, liebe diesen Overall. Es mac...  
4  Dieses Shirt ist aufgrund der verstellbaren Fr...

In [53]:
data = data[['Review Text', 'Recommended IND', 'Review Text DE']]
data.tail()

Review Text  Recommended IND  \
23481  I was very happy to snag this dress at such a ...                1   
23482  It reminds me of maternity clothes. soft, stre...                1   
23483  This fit well, but the top was very see throug...                0   
23484  I bought this dress for a wedding i have this ...                1   
23485  This dress in a lovely platinum is feminine an...                1   

                                          Review Text DE  
23481  Ich war sehr glücklich, dieses Kleid zu einem ...  
23482  Es erinnert mich an Umstandsmode. weiches, deh...  
23483  Das passte gut, aber das Top war sehr durchsic...  
23484  Ich habe dieses Kleid für eine Hochzeit gekauf...  
23485  Dieses Kleid in einem schönen Platin ist femin...

In [54]:
data.isna().sum()

Review Text           0
Recommended IND       0
Review Text DE     8703
dtype: int64

In [55]:
import spacy
import en_core_web_sm
from  spacy.lang.en.stop_words import STOP_WORDS
nlp = en_core_web_sm.load()
#nlp = spacy.load('en')

# To build a list of stop words for filtering
stopwords = list(STOP_WORDS)
print(stopwords)

['using', 'per', 'too', 'elsewhere', 'was', 'whatever', 'two', 'your', 'noone', 'will', 'hundred', 'own', "n't", 'and', 'everything', 'into', 'bottom', 'keep', 'are', 'how', 'thru', 'our', 'other', 'ever', 'hereafter', 'ca', 'after', 'because', 'regarding', 'put', 'hers', 'ours', 'had', 'her', 'mine', 'am', 'someone', 'off', 'somewhere', 'already', 'they', 'though', 'must', 'twelve', 'where', 'get', 'anyhow', 'he', 'latter', 'you', 'give', 'yourself', 'perhaps', 'she', 'nobody', 'latterly', 'can', 'itself', 'much', 'rather', 'up', 'unless', 'does', 'yours', "'ll", 'further', 'see', 'various', 'together', 'about', 'this', '’d', 'would', 'name', 'me', 'whereby', 'five', 'call', 'never', 'who', 'twenty', 'third', 'always', 'full', 'being', 'former', 'nowhere', 'may', 'show', 'something', 'towards', 'down', 'n‘t', 'one', 'therefore', 'or', 'have', 'above', 'no', "'d", 'at', 'there', '‘d', 'is', 'ourselves', 'whom', 'really', 'three', 'yourselves', "'re", 'indeed', 'among', 'than', 'wherein

In [56]:
import string
punctuations = string.punctuation
# Creating a Spacy Parser
from spacy.lang.en import English
parser = English()

In [57]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # return preprocessed list of tokens
    return mytokens

In [58]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [59]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [60]:
from sklearn.model_selection import train_test_split

X = data['Review Text'] # the features we want to analyze
ylabels = data['Recommended IND'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [61]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('cleaner', <__main__.predictors object at 0x7ff45707ec88>),
                ('vectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 t...\b\\w\\w+\\b',
                                 tokenizer=<function spacy_tokenizer at 0x7ff4310b3ea0>,
                                 vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
             

In [62]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [63]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()

In [64]:
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [65]:
# Splitting Data Set
from sklearn.model_selection import train_test_split

In [66]:
# Features and Labels
X = data['Review Text']
ylabels = data['Recommended IND']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

In [67]:
pipe_countvect = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])
# Fit our data
pipe_countvect.fit(X_train,y_train)

# Predicting with a test dataset
sample_prediction = pipe_countvect.predict(X_test)

# Prediction Results
# 1 = Positive review
# 0 = Negative review
for (sample,pred) in zip(X_test,sample_prediction):
    print(sample,"Prediction=>",pred)
    
# Accuracy
print("Accuracy: ",pipe_countvect.score(X_test,y_test))
print("Accuracy: ",pipe_countvect.score(X_test,sample_prediction))
# Accuracy
print("Accuracy: ",pipe_countvect.score(X_train,y_train))

I agree with the other reviewer that the color isn't as pink in person and more subtle. my thoughts below on some other things:
__________
pros:
- pockets! makes it so fun and chic with the overall style.
- the material is lovely and breathable. i didn't find it sheer at all! of course, i didn't wear it outside the dressing room. the material is thicker but it hangs on the body well.
- runs "mostly" tts. i only tried on the medium and it fit everywhere being slightly tighter in the chest. i Prediction=> 1
I have many sweaters but i bought this during the sale and it is very soft and comfortable. Prediction=> 1
Beautiful top but runs a whole size small. gave to my daughter who loved! Prediction=> 1
I loved these pants.they are so comfortable. however, after just wearing them twice, the threads of the leaves are fraying, and leaving lint on whatever i sit on. the leaves at the seat of the pants are now slowly disappearing. this after wearing twice to work! i imagine after a few more wear

In [68]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:", metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:", metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.9374201787994891
Logistic Regression Precision: 0.9473684210526315
Logistic Regression Recall: 0.9779048609305953


## Deutsch

In [12]:
kleidung_de = pd.read_csv('kleidung_de.csv')
kleidung_de.drop('Unnamed: 0', axis=1, inplace=True)
kleidung_de.head()

Review Text DE  Recommended IND
0    Absolut wundervoll - seidig und sexy und bequem                1
1  Liebe dieses Kleid! es ist sooo hübsch. Ich fa...                1
2  Ich hatte so große Hoffnungen auf dieses Kleid...                0
3  Ich liebe, liebe, liebe diesen Overall. Es mac...                1
4  Dieses Shirt ist aufgrund der verstellbaren Fr...                1

In [13]:
kleidung_de['Review Text DE'] = kleidung_de['Review Text DE'].astype(str)

In [15]:
!python -m spacy download de_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [18]:
import spacy
import de_core_news_sm
from  spacy.lang.de.stop_words import STOP_WORDS
nlp = de_core_news_sm.load()
#nlp = spacy.load('en')

# To build a list of stop words for filtering
stopwords = list(STOP_WORDS)
print(stopwords)

['jener', 'zehn', 'seit', 'siebten', 'zweiten', 'weiteren', 'muss', 'zehntes', 'ganze', 'konnten', 'wann', 'kommt', 'dass', 'wegen', 'selbst', 'dem', 'sechs', 'von', 'ihr', 'wieder', 'solcher', 'konnte', 'es', 'ins', 'acht', 'als', 'achte', 'heute', 'großes', 'gehen', 'willst', 'mag', 'sagte', 'vergangene', 'manche', 'sieben', 'zwar', 'ab', 'eigene', 'zugleich', 'ein', 'jedermann', 'beiden', 'grossen', 'dieser', 'vierte', 'eben', 'wollt', 'demselben', 'gekannt', 'gutes', 'je', 'weniges', 'vierten', 'siebte', 'deren', 'vergangenen', 'viele', 'jene', 'nie', 'bekannt', 'infolgedessen', 'seines', 'du', 'demgemäß', 'mochten', 'ihres', 'und', 'jahr', 'jetzt', 'nun', 'überhaupt', 'unsere', 'weiter', 'ging', 'er', 'mussten', 'kleine', 'diesem', 'diesen', 'nein', 'machte', 'wem', 'mehr', 'daran', 'ebenso', 'jahren', 'fünften', 'würden', 'sagt', 'leider', 'sich', 'kleines', 'manches', 'einmal', 'neunter', 'ohne', 'gehabt', 'neunten', 'haben', 'machen', 'jenem', 'anderen', 'würde', 'magst', 'jahr

In [19]:
import string
punctuations = string.punctuation
# Creating a Spacy Parser
from spacy.lang.de import German
parser = German()

In [20]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # return preprocessed list of tokens
    return mytokens

In [26]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

NameError: name 'TransformerMixin' is not defined

In [24]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

NameError: name 'CountVectorizer' is not defined

In [25]:
from sklearn.model_selection import train_test_split

X = kleidung_de['Review Text DE'] # the features we want to analyze
ylabels = kleidung_de['Recommended IND'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [ ]:
# Logistic Regression Classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

# Create pipeline using Bag of Words
pipe = Pipeline([('cleaner', predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)